In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
카드 추천 시스템 구현 - LightFM과 DeepFM 알고리즘 적용 (Google Colab 버전)
- LightFM: 하이브리드 추천 시스템(협업 필터링 + 컨텐츠 기반)
- DeepFM: 딥러닝 기반 추천 시스템(FM + Deep Neural Network)
- Hit Rate 평가지표 사용
"""

# ## 1. 필요한 라이브러리 설치 및 임포트

# %%
# 필요한 라이브러리 설치
#!pip install lightfm pandas numpy scikit-learn tensorflow scipy matplotlib tqdm

# %%
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from google.colab import files  # 파일 업로드를 위한 Colab 전용 기능
warnings.filterwarnings('ignore')

# LightFM 임포트 시도
try:
    from lightfm import LightFM
    from lightfm.evaluation import precision_at_k, recall_at_k
    LIGHTFM_AVAILABLE = True
    print("LightFM 라이브러리가 성공적으로 로드되었습니다.")
except ImportError:
    LIGHTFM_AVAILABLE = False
    print("경고: LightFM 라이브러리를 불러올 수 없습니다. LightFM 기능이 비활성화됩니다.")

# TensorFlow 임포트
try:
    import tensorflow as tf
    from tensorflow.keras import layers, Model, optimizers
    TENSORFLOW_AVAILABLE = True
    print("TensorFlow 라이브러리가 성공적으로 로드되었습니다.")
except ImportError:
    TENSORFLOW_AVAILABLE = False
    print("경고: TensorFlow 라이브러리를 불러올 수 없습니다. DeepFM 기능이 비활성화됩니다.")

# %% [markdown]
# ## 2. 데이터 업로드 및 확인
# 데이터 파일을 Google Colab에 업로드합니다. 코랩에서는 files.upload() 함수를 사용하여 파일을 업로드할 수 있습니다.

# %%
# Google Colab에 파일 업로드 (실행 후 파일 선택 대화상자에서 파일 선택)
print("cus_card.csv 파일을 업로드하세요.")
uploaded = files.upload()  # cus_card.csv 파일 업로드

print("\ncards.csv 파일을 업로드하세요.")
uploaded = files.upload()  # cards.csv 파일 업로드

print("\nbenefits.csv 파일을 업로드하세요.")
uploaded = files.upload()  # benefits.csv 파일 업로드

print("\nwoori_10pct_labeled.csv 파일을 업로드하세요.")
uploaded = files.upload()  # woori_10pct_labeled.csv 파일 업로드

# %%
# 데이터 로드
df_cus_card = pd.read_csv('cus_card.csv')
df_cards = pd.read_csv('cards.csv')
df_benefits = pd.read_csv('benefits.csv')
df_clusters = pd.read_csv('woori_10pct_labeled.csv')

# 'SEQ' 열이 있으면 'customer_id'로 이름 변경
if 'SEQ' in df_clusters.columns and 'customer_id' not in df_clusters.columns:
    df_clusters.rename(columns={'SEQ': 'customer_id'}, inplace=True)
    print("'SEQ' 열을 'customer_id'로 이름 변경했습니다.")

# 데이터 정보 출력
print(f"고객 수: {df_cus_card.shape[0]}")
print(f"카드 수: {df_cards.shape[0]}")
print(f"혜택 수: {df_benefits.shape[0]}")
print(f"클러스터 데이터 수: {df_clusters.shape[0]}")

# 클러스터 정보 확인
print("\n클러스터 분포:")
print(df_clusters['cluster'].value_counts())

# %% [markdown]
# ## 3. 데이터 전처리

# %%
# 고객별 보유 카드 정보 전처리
def preprocess_customer_card_data(df_cus_card, df_cards):
    # 고객별 보유 카드 목록 생성
    customer_cards = {}
    for _, row in df_cus_card.iterrows():
        customer_id = row['customer_id']
        cards = [card_name for col in df_cus_card.columns if '보유카드' in col
                for card_name in [row[col]] if pd.notna(card_name)]
        customer_cards[customer_id] = cards

    # 보유 카드 이름을 카드 ID로 매핑
    card_name_to_id = {}
    for _, row in df_cards.iterrows():
        card_name_to_id[row['card_name']] = row['card_id']

    # 고객-카드 상호작용 데이터 생성
    interactions = []
    for customer_id, cards in customer_cards.items():
        for card_name in cards:
            if card_name in card_name_to_id:
                interactions.append((customer_id, card_name_to_id[card_name]))

    df_interactions = pd.DataFrame(data=interactions, columns=['customer_id', 'card_id'])
    return df_interactions

# 카드 혜택 정보 전처리
def preprocess_card_benefits(df_benefits):
    # 카드별 혜택 텍스트 합치기
    card_benefits = df_benefits.groupby('card_id')['benefit_text'].apply(
        lambda x: ' '.join(x)).reset_index()
    return card_benefits

# 상호작용 데이터 생성
df_interactions = preprocess_customer_card_data(df_cus_card, df_cards)
print(f"상호작용 데이터 건수: {df_interactions.shape[0]}")

# 카드 혜택 정보 처리
df_card_benefits = preprocess_card_benefits(df_benefits)

# 소비 패턴과 클러스터 정보 병합
df_customer_profile = df_clusters.copy()

# 사용자와 아이템 인덱스 생성
customer_encoder = LabelEncoder()
card_encoder = LabelEncoder()

df_interactions['customer_idx'] = customer_encoder.fit_transform(
    df_interactions['customer_id'].values
)
df_interactions['card_idx'] = card_encoder.fit_transform(
    df_interactions['card_id'].values
)

# 고객 및 카드 수 계산
n_customers = df_interactions['customer_idx'].nunique()
n_cards = df_interactions['card_idx'].nunique()

print(f"전처리 후 고객 수: {n_customers}")
print(f"전처리 후 카드 수: {n_cards}")

# %% [markdown]
# ## 4. 모델링을 위한 데이터 준비

# %%
# 훈련 및 테스트 세트 분리
train_interactions, test_interactions = train_test_split(df_interactions, test_size=0.2, random_state=42)

# ID를 인덱스로 매핑하는 딕셔너리 생성
customer_id_to_idx = dict(zip(customer_encoder.classes_, range(len(customer_encoder.classes_))))
card_id_to_idx = dict(zip(card_encoder.classes_, range(len(card_encoder.classes_))))

# LightFM용 상호작용 행렬 생성
def create_interaction_matrix(df, n_users, n_items, user_col, item_col):
    user_indices = df[user_col].values
    item_indices = df[item_col].values
    values = np.ones(len(df), dtype=np.float32)

    return sparse.coo_matrix((values, (user_indices, item_indices)),
                          shape=(n_users, n_items), dtype=np.float32)

# LightFM용 행렬 생성
train_matrix = create_interaction_matrix(
    train_interactions,
    n_customers,
    n_cards,
    'customer_idx',
    'card_idx'
)
# coo_matrix를 csr_matrix로 변환 (인덱싱 가능하도록)
train_matrix = train_matrix.tocsr()

test_matrix = create_interaction_matrix(
    test_interactions,
    n_customers,
    n_cards,
    'customer_idx',
    'card_idx'
)
# coo_matrix를 csr_matrix로 변환 (인덱싱 가능하도록)
test_matrix = test_matrix.tocsr()

# 클러스터 ID 저장
cluster_ids = df_customer_profile['cluster'].unique()

# %% [markdown]
# ## 5. 특성(Feature) 행렬 생성

# %%
# 카드 특성 생성 (TF-IDF로 혜택 텍스트 벡터화)
tfidf = TfidfVectorizer(max_features=100)

# 카드 혜택 데이터 준비
card_texts = []
for idx in range(n_cards):
    card_id = card_encoder.inverse_transform([idx])[0]
    benefit_texts = df_card_benefits[df_card_benefits['card_id'] == card_id]['benefit_text'].tolist()
    if len(benefit_texts) > 0:
        card_texts.append(benefit_texts[0])
    else:
        card_texts.append("")

# TF-IDF 벡터화
card_features_matrix = tfidf.fit_transform(card_texts)
print(f"카드 특성 행렬 크기: {card_features_matrix.shape}")

# 사용자 특성 생성
# 소비 패턴 컬럼만 선택
feature_cols = [col for col in df_customer_profile.columns
               if col not in ['SEQ', 'customer_id', 'cluster']]

user_features = np.zeros((int(n_customers), len(feature_cols) + 1))  # +1 for cluster

for _, row in df_customer_profile.iterrows():
    if row['customer_id'] in customer_id_to_idx:
        idx = customer_id_to_idx[row['customer_id']]
        feature_values = row[feature_cols].tolist()
        user_features[idx, :-1] = feature_values
        user_features[idx, -1] = row['cluster']  # 클러스터 정보 추가

# 표준화
scaler = StandardScaler()
user_features = scaler.fit_transform(user_features)
user_features_matrix = sparse.csr_matrix(user_features)
print(f"사용자 특성 행렬 크기: {user_features_matrix.shape}")

# %% [markdown]
# ## 6. LightFM 모델 구현 및 훈련

# %%
if LIGHTFM_AVAILABLE:
    # LightFM 모델 평가 함수
    def calculate_lightfm_hit_rate(model, user_indices, full_test_matrix, full_train_matrix, n=10):
        """LightFM Hit Rate 계산"""
        hits = 0
        total_users = 0

        for user_idx in user_indices:
            try:
                # 테스트 세트의 실제 항목
                test_items = full_test_matrix[user_idx].indices
                if len(test_items) == 0:
                    continue

                # 훈련 세트에서 이미 상호작용한 항목
                train_items = full_train_matrix[user_idx].indices

                # 모든 아이템에 대한 예측 점수 계산 - 사용자 특성을 개별적으로 전달하지 않음
                scores = model.predict(
                    user_idx,
                    np.arange(n_cards),
                    user_features=None,  # 사용자 특성 매개변수 제거
                    item_features=card_features_matrix
                )

                # 훈련 세트의 항목 점수 낮추기
                scores[train_items] = -np.inf

                # 상위 N개 아이템 선택
                top_items = np.argsort(-scores)[:n]

                # 상위 N개 중 실제 아이템이 있는지 확인
                if np.intersect1d(top_items, test_items).size > 0:
                    hits += 1

                total_users += 1

            except Exception as e:
                print(f"사용자 {user_idx} 평가 중 오류: {str(e)}")
                continue

        hit_rate = hits / total_users if total_users > 0 else 0
        return hit_rate

    print("LightFM 클러스터별 모델 훈련 중...")
    # 클러스터별 모델과 평가 결과 저장
    lightfm_models = {}
    lightfm_hit_rates = {}

    for cluster in cluster_ids:
        print(f"\n클러스터 {cluster} LightFM 모델 훈련 중...")

        # 클러스터에 해당하는 사용자 인덱스 추출
        cluster_users = df_customer_profile[
            df_customer_profile['cluster'] == cluster]['customer_id'].tolist()
        cluster_user_indices = [customer_id_to_idx[user_id]
                            for user_id in cluster_users
                            if user_id in customer_id_to_idx]

        if len(cluster_user_indices) == 0:
            lightfm_hit_rates[cluster] = 0
            continue

        # 클러스터별 훈련 및 테스트 행렬 생성
        cluster_train_matrix = sparse.lil_matrix(train_matrix.shape)
        cluster_test_matrix = sparse.lil_matrix(test_matrix.shape)

        # 해당 클러스터 사용자의 데이터만 추출
        for user_idx in cluster_user_indices:
            cluster_train_matrix[user_idx] = train_matrix[user_idx]
            cluster_test_matrix[user_idx] = test_matrix[user_idx]

        # csr 형식으로 변환 (효율적인 연산을 위해)
        cluster_train_matrix = cluster_train_matrix.tocsr()
        cluster_test_matrix = cluster_test_matrix.tocsr()

        # LightFM 모델 초기화 및 훈련 - 클러스터별 별도 모델
        lightfm_model = LightFM(loss='warp', no_components=30, learning_rate=0.05, random_state=42)

        # 모델 훈련 (클러스터별 데이터로)
        lightfm_model.fit(
            cluster_train_matrix,
            user_features=None,
            item_features=card_features_matrix,
            epochs=30,
            num_threads=4,
            verbose=True
        )

        # 모델 저장
        lightfm_models[cluster] = lightfm_model

        # 클러스터별 모델 평가
        hit_rate = calculate_lightfm_hit_rate(
            lightfm_model,
            cluster_user_indices,
            cluster_test_matrix,
            cluster_train_matrix,
            n=10
        )
        lightfm_hit_rates[cluster] = hit_rate
        print(f"클러스터 {cluster} LightFM Hit Rate: {hit_rate:.4f}")
else:
    print("LightFM 라이브러리를 사용할 수 없어 훈련을 건너뜁니다.")
    lightfm_models = {}
    lightfm_hit_rates = {}

# %% [markdown]
# ## 7. DeepFM 모델 구현 및 훈련

# %%
if TENSORFLOW_AVAILABLE:
    print("\nDeepFM 클러스터별 모델 훈련 중...")

    # DeepFM 모델 구현
    class FMLayer(layers.Layer):
        def __init__(self, feature_dim, embedding_size=10, **kwargs):
            super(FMLayer, self).__init__(**kwargs)
            self.feature_dim = feature_dim
            self.embedding_size = embedding_size
            # 특성 차원이 너무 크면 임베딩 크기를 조정
            print(f"FMLayer 초기화: 특성 차원 = {feature_dim}, 임베딩 크기 = {embedding_size}")
            self.embedding_layer = layers.Embedding(feature_dim, embedding_size)

        def call(self, inputs):
            # 피처 범위
            feature_range = tf.range(self.feature_dim, dtype=tf.int32)

            # 임베딩
            embeddings = self.embedding_layer(feature_range)

            # 피처별 임베딩 적용
            feature_embeddings = tf.multiply(
                tf.expand_dims(inputs, axis=-1),
                embeddings
            )

            # FM 2차 상호작용 계산
            summed_square = tf.square(tf.reduce_sum(feature_embeddings, axis=1))
            squared_sum = tf.reduce_sum(tf.square(feature_embeddings), axis=1)
            # 리듀스 차원을 추가하여 스칼라로 변환
            fm_interaction = tf.reduce_sum(0.5 * tf.subtract(summed_square, squared_sum), axis=1, keepdims=True)
            return fm_interaction

    class DeepFM:
        def __init__(self, feature_dim, embedding_size=10, dnn_hidden_units=[64, 32, 16]):
            self.feature_dim = feature_dim
            self.embedding_size = embedding_size
            self.dnn_hidden_units = dnn_hidden_units
            self.model = self.build_model()

        def build_model(self):
            # 입력 레이어 - 실제 특성 차원 사용
            print(f"DeepFM 모델 구축: 입력 특성 차원 = {self.feature_dim}")
            inputs = layers.Input(shape=(self.feature_dim,))

            # FM 부분
            # 선형(1차) 부분
            linear_output = layers.Dense(1)(inputs)

            # 2차 상호작용 부분 - 커스텀 레이어 사용
            fm_output = FMLayer(self.feature_dim, self.embedding_size)(inputs)

            # DNN 부분
            dnn_output = inputs
            for units in self.dnn_hidden_units:
                dnn_output = layers.Dense(units, activation='relu')(dnn_output)
            dnn_output = layers.Dense(1)(dnn_output)

            # 최종 출력 - fm_output이 이미 (batch_size, 1) 형태로 출력됨
            outputs = layers.Add()([linear_output, fm_output, dnn_output])
            outputs = layers.Activation('sigmoid')(outputs)

            # 모델 컴파일
            model = Model(inputs=inputs, outputs=outputs)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            return model

    # DeepFM 모델용 데이터 준비
    def prepare_deepfm_data(df_interactions, df_customer_profile, card_features_matrix, card_encoder):
        # 전역 feature_cols 변수 사용
        global feature_cols, feature_dim
        print(f"prepare_deepfm_data 호출: feature_cols 길이 = {len(feature_cols)}")

        # 사용자-카드 특성 결합
        df = df_interactions.copy()

        # 카드 특성 추가
        df['card_type'] = df['card_id'].apply(lambda x: 1 if x >= 1000 else 0)  # 카드 유형(체크=0, 신용=1)

        # 사용자 특성 추가
        df = pd.merge(
            df,
            df_customer_profile[['customer_id', 'cluster', 'TOT_USE_AM']],
            on='customer_id',
            how='left'
        )

        # 카드 혜택 정보 추가 (단순화)
        df['has_benefits'] = df['card_id'].apply(
            lambda x: 1 if x in df_card_benefits['card_id'].values else 0
        )

        # 원-핫 인코딩
        clusters_onehot = pd.get_dummies(df['cluster'], prefix='cluster')
        df = pd.concat([df, clusters_onehot], axis=1)

        # 카드 ID를 카드 인덱스로 변환하여 TF-IDF 특성 추가
        card_id_to_idx = {card_id: idx for idx, card_id in enumerate(card_encoder.classes_)}

        # TF-IDF 특성 추출 및 추가
        tfidf_features = []
        for card_id in df['card_id']:
            if card_id in card_id_to_idx:
                card_idx = card_id_to_idx[card_id]
                # 해당 카드의 TF-IDF 벡터 추출
                tfidf_vector = card_features_matrix[card_idx].toarray().flatten()
                tfidf_features.append(tfidf_vector)
            else:
                # 매핑되지 않는 카드는 0 벡터로 처리
                tfidf_features.append(np.zeros(card_features_matrix.shape[1]))

        # TF-IDF 특성을 DataFrame으로 변환
        tfidf_df = pd.DataFrame(
            tfidf_features,
            columns=[f'tfidf_{i}' for i in range(card_features_matrix.shape[1])]
        )

        # 원본 데이터프레임과 TF-IDF 특성 결합
        df = pd.concat([df, tfidf_df], axis=1)

        # 모든 필요한 특성 컬럼이 데이터프레임에 있는지 확인
        for col in feature_cols:
            if col not in df.columns:
                df[col] = 0.0

        # 특성 추출 전에 feature_cols의 모든 컬럼이 df에 있는지 확인
        for col in feature_cols:
            if col not in df.columns:
                print(f"경고: 컬럼 '{col}'이 데이터프레임에 없습니다. 0으로 채웁니다.")
                df[col] = 0.0

        # 특성 추출 - feature_cols에 정의된 컬럼만 사용
        features = df[feature_cols]

        # 디버깅 정보 출력
        print(f"prepare_deepfm_data: 특성 컬럼 수 = {len(feature_cols)}, 특성 행렬 크기 = {features.shape}")

        # 표준화
        scaler = StandardScaler()
        features = scaler.fit_transform(features)

        # 모든 상호작용은 긍정적(1)으로 간주
        labels = np.ones(len(df))

        return features, labels, df

    # DeepFM 모델 평가 함수 정의
    def calculate_deepfm_hit_rate(test_data, model, n=10):
        """DeepFM Hit Rate 계산 - 전체 카드에 대해 예측하고 평가"""
        hits = 0
        total_users = 0

        # 사용자별로 그룹화
        unique_users = test_data['customer_id'].unique()

        for customer_id in unique_users:
            # 테스트 데이터에서 실제 상호작용한 카드
            test_items = test_data[test_data['customer_id'] == customer_id]['card_id'].tolist()
            if len(test_items) == 0:
                continue

            # 훈련 데이터에서 이미 사용자가 가진 카드
            user_items = train_interactions[train_interactions['customer_id'] == customer_id]['card_id'].tolist()

            # 사용자 프로필 정보 가져오기
            user_profile = df_customer_profile[df_customer_profile['customer_id'] == customer_id]
            if user_profile.empty:
                continue

            total_users += 1

            # 전체 카드에 대해 예측 수행
            all_cards = df_cards['card_id'].values
            candidate_cards = [card for card in all_cards if card not in user_items]

            # 사용자 특성 및 카드 특성 배치 준비
            batch_features = []
            for card_id in candidate_cards:
                # 카드 특성 생성
                card_type = 1 if card_id >= 1000 else 0
                has_benefits = 1 if card_id in df_card_benefits['card_id'].values else 0

                # 사용자 클러스터 정보
                cluster = user_profile['cluster'].iloc[0]

                # 특성 딕셔너리 생성 (기본 특성)
                features_dict = {
                    'card_type': card_type,
                    'TOT_USE_AM': user_profile['TOT_USE_AM'].iloc[0],
                    'has_benefits': has_benefits
                }

                # 클러스터에 대한 원-핫 인코딩 추가
                for c in cluster_ids:
                    col_name = f'cluster_{c}'
                    features_dict[col_name] = 1 if cluster == c else 0

                # TF-IDF 특성 추가
                if card_id in card_id_to_idx:
                    card_idx = card_id_to_idx[card_id]
                    tfidf_vector = card_features_matrix[card_idx].toarray().flatten()
                    for i, val in enumerate(tfidf_vector):
                        features_dict[f'tfidf_{i}'] = val
                else:
                    for i in range(card_features_matrix.shape[1]):
                        features_dict[f'tfidf_{i}'] = 0.0

                # 특성 벡터 생성 - feature_cols에 정의된 모든 컬럼 사용
                feature_vector = []
                for col in feature_cols:
                    feature_vector.append(features_dict.get(col, 0.0))

                # 벡터 길이 확인
                if len(feature_vector) != model.feature_dim:
                    # 필요한 경우 벡터 길이 조정
                    if len(feature_vector) < model.feature_dim:
                        # 부족한 경우 0으로 채움
                        feature_vector.extend([0.0] * (model.feature_dim - len(feature_vector)))
                    else:
                        # 넘치는 경우 자름
                        feature_vector = feature_vector[:model.feature_dim]

                batch_features.append(feature_vector)

            # 배치로 예측
            if batch_features:
                batch_features_array = np.array(batch_features)
                batch_scores = model.model.predict(batch_features_array, verbose=0).flatten()

                # 카드 ID와 점수 연결
                card_scores = list(zip(candidate_cards, batch_scores))

                # 점수 기준으로 상위 N개 카드 선택
                card_scores.sort(key=lambda x: x[1], reverse=True)
                recommended_items = [card_id for card_id, _ in card_scores[:n]]

                # 상위 N개 중 실제 테스트 아이템이 있는지 확인
                if len(np.intersect1d(recommended_items, test_items)) > 0:
                    hits += 1

        hit_rate = hits / total_users if total_users > 0 else 0
        return hit_rate

    # 클러스터별 모델과 평가 결과 저장
    deepfm_models = {}
    deepfm_hit_rates = {}

    # DeepFM 모델의 특성 컬럼 목록 저장 (다른 함수에서 재사용)
    feature_cols = ['card_type', 'TOT_USE_AM', 'has_benefits'] + \
                  [f'cluster_{c}' for c in cluster_ids] + \
                  [f'tfidf_{i}' for i in range(card_features_matrix.shape[1])]

    # 전역 변수로 feature_dim 선언 (다른 함수에서 재사용)
    global feature_dim
    feature_dim = len(feature_cols)
    print(f"feature_cols 정의: {len(feature_cols)}개 특성")

    for cluster in cluster_ids:
        print(f"\n클러스터 {cluster} DeepFM 모델 훈련 중...")

        # 클러스터에 해당하는 사용자 목록 추출
        cluster_users = df_customer_profile[
            df_customer_profile['cluster'] == cluster]['customer_id'].tolist()

        if len(cluster_users) == 0:
            deepfm_hit_rates[cluster] = 0
            continue

        # 클러스터별 훈련 및 테스트 데이터 추출
        cluster_train_interactions = train_interactions[
            train_interactions['customer_id'].isin(cluster_users)
        ]

        cluster_test_interactions = test_interactions[
            test_interactions['customer_id'].isin(cluster_users)
        ]

        if len(cluster_train_interactions) == 0 or len(cluster_test_interactions) == 0:
            deepfm_hit_rates[cluster] = 0
            continue

        # 클러스터별 훈련 데이터 준비
        train_features, train_labels, train_df = prepare_deepfm_data(
            cluster_train_interactions, df_customer_profile, card_features_matrix, card_encoder
        )

        test_features, test_labels, test_df = prepare_deepfm_data(
            cluster_test_interactions, df_customer_profile, card_features_matrix, card_encoder
        )

        # 특성 차원 확인 및 조정
        if train_features.shape[1] != feature_dim:
            print(f"경고: 특성 차원 불일치! 훈련 데이터 차원({train_features.shape[1]})이 모델 입력 차원({feature_dim})과 다릅니다.")

            # 더 작은 차원으로 맞추기
            min_dim = min(train_features.shape[1], feature_dim)
            if train_features.shape[1] > min_dim:
                train_features = train_features[:, :min_dim]
                test_features = test_features[:, :min_dim]

            cluster_feature_dim = min_dim
        else:
            cluster_feature_dim = feature_dim

        # 클러스터별 DeepFM 모델 초기화 및 훈련
        cluster_deepfm_model = DeepFM(
            feature_dim=cluster_feature_dim,
            embedding_size=16,
            dnn_hidden_units=[128, 64, 32]
        )

        # 조기 종료 콜백 정의
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )

        # 모델 훈련
        print(f"클러스터 {cluster} DeepFM 모델 훈련 시작...")
        cluster_deepfm_model.model.fit(
            train_features,
            train_labels,
            epochs=30,
            batch_size=256,
            validation_split=0.1,
            callbacks=[early_stopping],
            verbose=0
        )
        print(f"클러스터 {cluster} DeepFM 모델 훈련 완료")

        # 모델 저장
        deepfm_models[cluster] = cluster_deepfm_model

        # 클러스터별 모델 평가
        hit_rate = calculate_deepfm_hit_rate(test_df, cluster_deepfm_model)
        deepfm_hit_rates[cluster] = hit_rate
        print(f"클러스터 {cluster} DeepFM Hit Rate: {hit_rate:.4f}")
else:
    print("TensorFlow 라이브러리를 사용할 수 없어 훈련을 건너뜁니다.")
    deepfm_hit_rates = {}
    deepfm_models = {}

# %% [markdown]
# ## 8. 클러스터별 최적 알고리즘 선택

# %%
# 클러스터별 최적 알고리즘 선택
print("\n클러스터별 최적 알고리즘 선택:")

best_model_by_cluster = {}
for cluster in cluster_ids:
    lightfm_hit_rate = lightfm_hit_rates.get(cluster, 0)
    deepfm_hit_rate = deepfm_hit_rates.get(cluster, 0)

    if lightfm_hit_rate > deepfm_hit_rate:
        best_model = "LightFM"
        hit_rate = lightfm_hit_rate
    else:
        best_model = "DeepFM"
        hit_rate = deepfm_hit_rate

    best_model_by_cluster[cluster] = (best_model, hit_rate)
    print(f"클러스터 {cluster}: {best_model} (Hit Rate: {hit_rate:.4f})")

# %% [markdown]
# ## 9. 고객별 추천 카드 생성

# %%
# 고객별 추천 카드 생성 함수
def get_top_n_recommendations(model_type, customer_idx, customer_id, n=5):
    """상위 N개 추천 카드 생성"""
    # 사용자 클러스터 확인
    user_profile = df_customer_profile[df_customer_profile['customer_id'] == customer_id]
    if user_profile.empty:
        return []

    user_cluster = user_profile['cluster'].iloc[0]

    if model_type == "LightFM" and user_cluster in lightfm_models:
        # 클러스터별 LightFM 모델 사용
        lightfm_model = lightfm_models[user_cluster]

        # 모든 카드에 대한 점수 예측
        scores = lightfm_model.predict(
            customer_idx,
            np.arange(n_cards),
            user_features=None,
            item_features=card_features_matrix
        )

        # 이미 보유한 카드 제외
        user_items = train_matrix[customer_idx].indices
        scores[user_items] = -np.inf

        # 상위 n개 선택
        top_items = np.argsort(-scores)[:n]

        # 카드 ID로 변환
        top_cards = card_encoder.inverse_transform(top_items)
        return top_cards

    elif model_type == "DeepFM" and user_cluster in deepfm_models:
        # 클러스터별 DeepFM 모델 사용
        deepfm_model = deepfm_models[user_cluster]

        # 추천할 수 있는 모든 카드 목록
        all_cards = df_cards['card_id'].values

        # 이미 보유한 카드 제외
        user_cards = train_interactions[
            train_interactions['customer_id'] == customer_id]['card_id'].tolist()
        candidate_cards = [card for card in all_cards if card not in user_cards]

        # 사용자 특성 및 카드 특성 배치 준비
        batch_features = []
        for card_id in candidate_cards:
            # 카드 특성 생성
            card_type = 1 if card_id >= 1000 else 0
            has_benefits = 1 if card_id in df_card_benefits['card_id'].values else 0

            # 사용자 클러스터 정보
            cluster = user_profile['cluster'].iloc[0]

            # 특성 딕셔너리 생성 (기본 특성)
            features_dict = {
                'card_type': card_type,
                'TOT_USE_AM': user_profile['TOT_USE_AM'].iloc[0],
                'has_benefits': has_benefits
            }

            # 클러스터에 대한 원-핫 인코딩 수동 추가
            for c in cluster_ids:
                col_name = f'cluster_{c}'
                features_dict[col_name] = 1 if cluster == c else 0

            # TF-IDF 특성 추가
            if card_id in card_id_to_idx:
                card_idx = card_id_to_idx[card_id]
                tfidf_vector = card_features_matrix[card_idx].toarray().flatten()
                for i, val in enumerate(tfidf_vector):
                    features_dict[f'tfidf_{i}'] = val
            else:
                for i in range(card_features_matrix.shape[1]):
                    features_dict[f'tfidf_{i}'] = 0.0

            # 특성 벡터 생성 - feature_cols에 정의된 모든 컬럼 사용
            feature_vector = []
            for col in feature_cols:
                feature_vector.append(features_dict.get(col, 0.0))

            # 벡터 길이 확인
            if len(feature_vector) != deepfm_model.feature_dim:
                # 필요한 경우 벡터 길이 조정
                if len(feature_vector) < deepfm_model.feature_dim:
                    # 부족한 경우 0으로 채움
                    feature_vector.extend([0.0] * (deepfm_model.feature_dim - len(feature_vector)))
                else:
                    # 넘치는 경우 자름
                    feature_vector = feature_vector[:deepfm_model.feature_dim]

            batch_features.append(feature_vector)

        # 배치로 예측
        if batch_features:
            batch_features_array = np.array(batch_features)
            batch_scores = deepfm_model.model.predict(batch_features_array, verbose=0).flatten()

            # 카드 ID와 점수 연결
            card_scores = list(zip(candidate_cards, batch_scores))

            # 점수 기준으로 상위 N개 카드 선택
            card_scores.sort(key=lambda x: x[1], reverse=True)
            top_cards = [card_id for card_id, _ in card_scores[:n]]
            return top_cards

        return []

    return []

# 고객별 추천 결과 저장
print("\n고객별 추천 카드 생성 중...")
recommendations = {}

for _, customer in tqdm(df_customer_profile.iterrows(), total=len(df_customer_profile)):
    customer_id = customer['customer_id']
    if customer_id not in customer_id_to_idx:
        continue

    customer_idx = customer_id_to_idx[customer_id]
    cluster = customer['cluster']

    # 클러스터에 따른 최적 모델 선택
    if cluster in best_model_by_cluster:
        best_model = best_model_by_cluster[cluster][0]

        # 추천 카드 생성
        top_cards = get_top_n_recommendations(best_model, customer_idx, customer_id, n=5)

        # 카드 이름으로 변환
        card_names = [df_cards[df_cards['card_id'] == card_id]['card_name'].iloc[0]
                     for card_id in top_cards if card_id in df_cards['card_id'].values.tolist()]

        recommendations[customer_id] = card_names

# %% [markdown]
# ## 10. 결과 저장 및 요약

# %%
# 추천 결과를 DataFrame으로 변환
recommendations_list = []
for customer_id, card_names in recommendations.items():
    for i, card_name in enumerate(card_names, 1):
        recommendations_list.append({
            'customer_id': customer_id,
            'rank': i,
            'card_name': card_name
        })

df_recommendations = pd.DataFrame(recommendations_list)

# 결과 저장
df_recommendations.to_csv('card_recommendations.csv', index=False, encoding='utf-8-sig')
print("추천 결과가 card_recommendations.csv 파일로 저장되었습니다.")

# 결과 다운로드
from google.colab import files
files.download('card_recommendations.csv')

# 구현 결과 요약
print("\n=== 카드 추천 시스템 구현 결과 ===")
print(f"총 고객 수: {df_cus_card.shape[0]}")
print(f"총 추천 고객 수: {len(recommendations)}")
print(f"클러스터 수: {len(cluster_ids)}")

print("\n클러스터별 최적 알고리즘:")
for cluster, (model, hit_rate) in best_model_by_cluster.items():
    print(f"클러스터 {cluster}: {model} (Hit Rate: {hit_rate:.4f})")

print("\n추천 샘플 (5명):")
sample_recommendations = list(recommendations.items())[:5]
for customer_id, card_names in sample_recommendations:
    print(f"고객 ID: {customer_id}")
    for i, card_name in enumerate(card_names, 1):
        print(f"  {i}. {card_name}")
    print()

LightFM 라이브러리가 성공적으로 로드되었습니다.
TensorFlow 라이브러리가 성공적으로 로드되었습니다.
cus_card.csv 파일을 업로드하세요.


Saving cus_card.csv to cus_card (1).csv

cards.csv 파일을 업로드하세요.


Saving cards.csv to cards (1).csv

benefits.csv 파일을 업로드하세요.


Saving benefits.csv to benefits (1).csv

woori_10pct_labeled.csv 파일을 업로드하세요.


Saving woori_10pct_labeled.csv to woori_10pct_labeled (1).csv
'SEQ' 열을 'customer_id'로 이름 변경했습니다.
고객 수: 45955
카드 수: 200
혜택 수: 1185
클러스터 데이터 수: 45955

클러스터 분포:
cluster
1    16999
2    11996
7     6916
4     4325
6     1645
3     1595
0     1428
5     1051
Name: count, dtype: int64
상호작용 데이터 건수: 91859
전처리 후 고객 수: 45955
전처리 후 카드 수: 198
카드 특성 행렬 크기: (198, 100)
사용자 특성 행렬 크기: (45955, 12)
LightFM 클러스터별 모델 훈련 중...

클러스터 1 LightFM 모델 훈련 중...


...

  0%|          | 0/45955 [00:00<?, ?it/s]

추천 결과가 card_recommendations.csv 파일로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


=== 카드 추천 시스템 구현 결과 ===
총 고객 수: 45955
총 추천 고객 수: 45955
클러스터 수: 8

클러스터별 최적 알고리즘:
클러스터 1: LightFM (Hit Rate: 0.0796)
클러스터 2: LightFM (Hit Rate: 0.0817)
클러스터 5: DeepFM (Hit Rate: 0.0701)
클러스터 7: LightFM (Hit Rate: 0.0724)
클러스터 4: LightFM (Hit Rate: 0.0793)
클러스터 3: LightFM (Hit Rate: 0.0885)
클러스터 0: LightFM (Hit Rate: 0.0717)
클러스터 6: LightFM (Hit Rate: 0.0753)

추천 샘플 (5명):
고객 ID: HSR6P43LIMPS12N0UZZ9
  1. zgm.the pay카드
  2. 신한카드 The BEST-F
  3. 카드의정석 EVERY MILE SKYPASS
  4. 트래블로그 체크카드
  5. the Green Edition2

고객 ID: 9GDU0YYKZ0EKNHIALX39
  1. 신한카드 플리 체크(산리오캐릭터즈)
  2. 신한카드 플리(산리오캐릭터즈)
  3. 삼성 iD EV 카드
  4. 마일앤조이카드(아시아나)
  5. taptap DIGITAL

고객 ID: AKOKJ7B1KBN723LTNHT8
  1. 노리2 체크카드(Play)
  2. 노리2 체크카드(Global)
  3. 케이뱅크 플러스 체크카드
  4. 트래블페이 충전카드
  5. 토스뱅크카드

고객 ID: F4XRBTIB1LNXY8MQXGZY
  1. the Red Edition5
  2. 카카오페이 체크카드
  3. PAYCO 포인트 카드
  4. 메리어트 본보이™ 더 베스트 신한카드
  5. 네이버페이 머니카드

고객 ID: 6N6OZ8CANCIEI11MA6YP
  1. 올바른 FLEX 카드
  2. NH올원 파이카드
  3. 올바른POINT체크카드
  4. NH올원 Shopping&11번가카드(R2타입)
